## Capstone Notebook  for collaboration restricted to non sensitive capstone activities
<a id = 'section-1'></a>
<a id = 'section-1'></a>
<a id = 'section-1'></a>
<a id = 'section-1'></a>
<a id = 'section-1'></a>
<a id = 'section-1'></a>



In [1]:
import numpy as np

In [2]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
!pip install bs4

  Stored in directory: /home/dsxuser/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [4]:

import urllib

toronto_zippc_uri = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
try:
    net_file = urllib.request.urlopen(toronto_zippc_uri)
    buffer = net_file.read() 
except IOError as ioe:
    print(ioe)

In [5]:
import pandas as pd
import re

def print_df(df):
    with pd.option_context('display.expand_frame_repr',False,'display.max_colwidth',47):
        print(df)

def table_to_zip_df(html_table_tag):
    rows_header = [re.compile('<[tdh]*>\s*[<a]*(["\s\w=/]*[>]*?["\s\w=/]*)[</a>]*\s*</[tdh]*>'),
                   re.compile('<tr>\s*<[tdh\s\w=/"]*>[\s\S]*</[tdh]*>\s*</tr>')]
                  
    rows = []
    
    header_horizontal_slice = rows_header[1].findall(str(html_table_tag))[0].split('</tr>')
    
    for index in range(len(header_horizontal_slice)):
                  header_horizontal_slice[index] = header_horizontal_slice[index].replace('<tr>','')
    
                  rows.insert(index,rows_header[0].findall(header_horizontal_slice[index]))
    for row_index in range (len(rows)):
        for col_index in range(len(rows[row_index])):
                               rows[row_index][col_index] = rows[row_index][col_index].replace('\n','')
                               rows[row_index][col_index] = re.sub('[\s\S]*>','',rows[row_index][col_index])

    zip_df = pd.DataFrame(data = rows[1:],columns = rows[0])
    return zip_df
    
    
from bs4 import BeautifulSoup

soup = BeautifulSoup(buffer,'html.parser')    
for table in soup.find_all('table'):
    table_df = table_to_zip_df(table)
    print_df(table_df) 
    break

    Postcode           Borough                                   Neighbourhood
0        M1A      Not assigned                                    Not assigned
1        M2A      Not assigned                                    Not assigned
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
7        M7A      Not assigned                                            None
8        M8A      Not assigned                                    Not assigned
9        M9A  Downtown Toronto                                            None
10       M1B              None                                            None
11       M1B              None                      

In [6]:
table_df = table_df[table_df['Borough'].isna() != True]
print_df(table_df)

    Postcode           Borough                                   Neighbourhood
0        M1A      Not assigned                                    Not assigned
1        M2A      Not assigned                                    Not assigned
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
7        M7A      Not assigned                                            None
8        M8A      Not assigned                                    Not assigned
9        M9A  Downtown Toronto                                            None
12       M2B      Not assigned                                    Not assigned
13       M3B        North York                      

In [7]:
table_df = table_df[table_df['Borough'] != 'Not assigned']
print_df(table_df)

    Postcode           Borough                                   Neighbourhood
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
9        M9A  Downtown Toronto                                            None
13       M3B        North York                                 Don Mills North
14       M4B         East York                                            None
15       M4B         East York                                            None
16       M5B  Downtown Toronto                                         Ryerson
17       M5B  Downtown Toronto                                 Garden District
18       M6B        North York                      

In [8]:
for row in range(table_df.shape[0]):
    if table_df.iloc[row,2] is None: 
        table_df.iloc[row,2] = table_df.iloc[row,1]
    elif table_df.iloc[row,2] == 'Not Assigned':
        table_df.iloc[row,2] = table_df.iloc[row,1]       

print_df(table_df)        

    Postcode           Borough                                   Neighbourhood
2        M3A        North York                                       Parkwoods
3        M4A        North York                                Victoria Village
4        M5A  Downtown Toronto                                    Harbourfront
5        M6A        North York                                Lawrence Heights
6        M6A        North York                                  Lawrence Manor
9        M9A  Downtown Toronto                                Downtown Toronto
13       M3B        North York                                 Don Mills North
14       M4B         East York                                       East York
15       M4B         East York                                       East York
16       M5B  Downtown Toronto                                         Ryerson
17       M5B  Downtown Toronto                                 Garden District
18       M6B        North York                      

In [9]:

table_df = table_df.transform({'Postcode': lambda x : x,'Borough':lambda x : " "+ x +",",'Neighbourhood': lambda x : " " + x + ","})

table_df = table_df.groupby(table_df['Postcode']).sum()

table_df = table_df.reset_index()  
print_df(table_df)


   Postcode                                         Borough                                   Neighbourhood
0       M1E                                      Guildwood,                                      Guildwood,
1       M1H                                      Cedarbrae,                                      Cedarbrae,
2       M1J                            Scarborough Village,                            Scarborough Village,
3       M1K                  East Birchmount Park, Ionview,                  East Birchmount Park, Ionview,
4       M1L                                       Clairlea,                                       Clairlea,
5       M1M           Cliffcrest, Scarborough Village West,           Cliffcrest, Scarborough Village West,
6       M1N                    Birch Cliff, Cliffside West,                    Birch Cliff, Cliffside West,
7       M1P           Dorset Park, Scarborough Town Centre,           Dorset Park, Scarborough Town Centre,
8       M1T                 

In [10]:
def rem_dupl_ent_post_sum(x):
        for index in range(x.shape[0]):
            val_str = x.iloc[index]
            val_list = val_str.split(',')
            repl_dict = {}
            repl_str = ''
            for value in val_list:
                repl_dict.update({value:''})
                
            for key_val,val in repl_dict.items():
                if len(repl_str) == 0:
                    repl_str = key_val
                else:
                    repl_str = repl_str + ',' + key_val
            x.iloc[index] = repl_str
        return x

In [11]:
tabl_df = table_df.transform({'Postcode' : lambda x : x,'Borough': rem_dupl_ent_post_sum,'Neighbourhood': rem_dupl_ent_post_sum})
print_df(table_df)

   Postcode                                 Borough                                   Neighbourhood
0       M1E                              Guildwood,                                      Guildwood,
1       M1H                              Cedarbrae,                                      Cedarbrae,
2       M1J                    Scarborough Village,                            Scarborough Village,
3       M1K          East Birchmount Park, Ionview,                  East Birchmount Park, Ionview,
4       M1L                               Clairlea,                                       Clairlea,
5       M1M   Cliffcrest, Scarborough Village West,           Cliffcrest, Scarborough Village West,
6       M1N            Birch Cliff, Cliffside West,                    Birch Cliff, Cliffside West,
7       M1P   Dorset Park, Scarborough Town Centre,           Dorset Park, Scarborough Town Centre,
8       M1T               Clarks Corners, Sullivan,                       Clarks Corners, Sullivan,


In [12]:
!pip install pgeocode

In [13]:
import pgeocode


nomi = pgeocode.Nominatim('ca')

df_pc = nomi.query_postal_code(table_df['Postcode'].to_list())
df_pc = df_pc[['postal_code','latitude','longitude']]
df_pc.columns = ['Postcode','latitude','longitude']
df_pc

table_df = pd.merge(table_df,df_pc)
            
table_df = table_df.dropna()
print_df(table_df)

   Postcode                                 Borough                                   Neighbourhood  latitude  longitude
0       M1E                              Guildwood,                                      Guildwood,   43.7678   -79.1866
1       M1H                              Cedarbrae,                                      Cedarbrae,   43.7686   -79.2389
2       M1J                    Scarborough Village,                            Scarborough Village,   43.7464   -79.2323
3       M1K          East Birchmount Park, Ionview,                  East Birchmount Park, Ionview,   43.7298   -79.2639
4       M1L                               Clairlea,                                       Clairlea,   43.7122   -79.2843
5       M1M   Cliffcrest, Scarborough Village West,           Cliffcrest, Scarborough Village West,   43.7247   -79.2312
6       M1N            Birch Cliff, Cliffside West,                    Birch Cliff, Cliffside West,   43.6952   -79.2646
7       M1P   Dorset Park, Scarb

In [14]:
print(table_df.shape)

(92, 5)


!openssl s_client -showcerts -servername archive.nyu.edu -connect archive.nyu.edu:443 > cacert.pem

!openssl x509 -inform PEM -in cacert.pem -text -out certdata

cat certdata

In [15]:
import urllib

ny_ziph_uri = 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm'
try:
    net_file = urllib.request.urlopen(ny_ziph_uri)
    buffer = net_file.read() 
except IOError as ioe:
    print(ioe)

In [16]:
import pandas as pd
import re
import copy

def print_df(df):
    with pd.option_context('display.expand_frame_repr',False,'display.max_colwidth',47):
        print(df)

def table_to_zip_df(html_table_tag):
    rows_header = [re.compile('<[tdh]*(?P<attributes>["\s\w\d=/,.-]*)>\s*[<a]*(["\s\w\d=/,.-]*[>]*?["\s\w\d=/,.-]*)[</a>]*\s*</[tdh]*>'),
                   re.compile('<tr>\s*<[tdh"\s\w\d=/,.-]*>[\s\S]*</[tdh]*>\s*</tr>')]
    
    attribute = [re.compile('(?<=headers=)["\w\d=/,.-]*|(?<=id=)["\w\d=/,.-]*')]
                  
    rows = []
    
    header_horizontal_slice = rows_header[1].findall(str(html_table_tag))[0].split('</tr>')

    for index in range(len(header_horizontal_slice)):
                  header_horizontal_slice[index] = header_horizontal_slice[index].replace('<tr>','')
    
                  rows.insert(index,rows_header[0].findall(header_horizontal_slice[index]))

    col_len = len(rows[0])
    
    if type(rows[0][0]) == tuple:
        for row_index in range (len(rows)):
            if len(rows[row_index]) != col_len: 
                row_data_holder = []
                for index in range(col_len):
                    row_data_holder.append('')
                for index in range (len(rows[row_index])):
                    row_data_holder[index] = rows[row_index][index]
                rows[row_index] = copy.deepcopy(row_data_holder)
                col_index = 0
                while col_index + 1  < col_len:
                    orig_col_index = col_index
                    data = row_data_holder[orig_col_index]
                    if data == '' :
                        col_index = col_index +1
                        continue
                    header_str = attribute[0].findall(data[0])[0]
                    header_num = re.search('header(\d)',header_str).group(1)
                    header_num = int(header_num)
                    rows[row_index][header_num-1] = data[1]
                    if rows[row_index][orig_col_index] == row_data_holder[orig_col_index]:
                        rows[row_index][orig_col_index] = ''
                    row_data_holder[orig_col_index] = ''
                    col_index = orig_col_index + 1                      
            else:
                for col_index in range(len(rows[row_index])):
                    rows[row_index][col_index] = rows[row_index][col_index][1] #.replace('\n','')
                    #rows[row_index][col_index] = re.sub('[\s\S]*>','',rows[row_index][col_index][1]) 
    elif type(rows[0][0]) == str:
        for row_index in range (len(rows)):
            for col_index in range(len(rows[row_index])):
                rows[row_index][col_index] = rows[row_index][col_index].replace('\n','')
                rows[row_index][col_index] = re.sub('[\s\S]*>','',rows[row_index][col_index])                   
    zip_df = pd.DataFrame(data = rows[1:],columns = rows[0])
    return zip_df
    
    
from bs4 import BeautifulSoup

soup = BeautifulSoup(buffer,'html.parser') 
for table in soup.find_all('table'):
    ziph_ny_df = table_to_zip_df(table)
    break
ziph_ny_df

,Borough,Neighborhood,ZIP Codes
0,Bronx,Central Bronx,"10453, 10457, 10460"
1,,Bronx Park and Fordham,"10458, 10467, 10468"
2,,High Bridge and Morrisania,"10451, 10452, 10456"
3,,Hunts Point and Mott Haven,"10454, 10455, 10459, 10474"
4,,Kingsbridge and Riverdale,"10463, 10471"
5,,Northeast Bronx,"10466, 10469, 10470, 10475"
6,,Southeast Bronx,"10461, 10462,10464, 10465, 10472, 10473"
7,Brooklyn,Central Brooklyn,"11212, 11213, 11216, 11233, 11238"
8,,Southwest Brooklyn,"11209, 11214, 11228"
9,,Borough Park,"11204, 11218, 11219, 11230"


In [17]:
df = ziph_ny_df[ziph_ny_df['ZIP Codes'] != '']
df = df.replace(to_replace = '',value = None)
df= df.fillna(method = 'ffill')
df
index_list = []
df_new = None 
for index in range(df.shape[0]):
    zip_list = df['ZIP Codes'][index].split(',')
    if len(zip_list) > 1:
        for item in  zip_list:
            new_d = pd.DataFrame( data = [[df.loc[index,'Borough'],df.loc[index,'Neighborhood'],item]], columns = df.columns.to_list() )
            
            if df_new is None:
                df_new = pd.DataFrame(new_d)
            else:
                df_new = pd.merge(df_new,new_d,how='outer')
            index_list.append(index)
df.drop(labels = index_list, inplace=True)

df = pd.merge(df,df_new,how = 'outer')
df

,Borough,Neighborhood,ZIP Codes
0,Staten Island,Mid-Island,10314
1,Bronx,Central Bronx,10453
2,Bronx,Central Bronx,10457
3,Bronx,Central Bronx,10460
4,Bronx,Bronx Park and Fordham,10458
5,Bronx,Bronx Park and Fordham,10467
6,Bronx,Bronx Park and Fordham,10468
7,Bronx,High Bridge and Morrisania,10451
8,Bronx,High Bridge and Morrisania,10452
9,Bronx,High Bridge and Morrisania,10456


In [18]:
df = df.groupby(df['ZIP Codes']).sum()
df

,Borough,Neighborhood
ZIP Codes,,
10003,Manhattan,Lower East Side
10005,Manhattan,Lower Manhattan
10006,Manhattan,Lower Manhattan
10007,Manhattan,Lower Manhattan
10009,Manhattan,Lower East Side
10011,Manhattan,Chelsea and Clinton
10013,Manhattan,Greenwich Village and Soho
10014,Manhattan,Greenwich Village and Soho
10016,Manhattan,Gramercy Park and Murray Hill


In [19]:
df = df.reset_index()

In [20]:
import pgeocode


nomi = pgeocode.Nominatim('us')

df_zp = nomi.query_postal_code(df['ZIP Codes'].to_list())
df_zp = df_zp[['postal_code','latitude','longitude']]
df_zp.columns = ['ZIP Codes','latitude','longitude']

df = pd.merge(df,df_zp)

df = df.dropna()
df

,ZIP Codes,Borough,Neighborhood,latitude,longitude
135,10001,Manhattan,Chelsea and Clinton,40.7484,-73.9967
136,10002,Manhattan,Lower East Side,40.7152,-73.9877
137,10004,Manhattan,Lower Manhattan,40.7143,-74.0060
138,10010,Manhattan,Gramercy Park and Murray Hill,40.7375,-73.9813
139,10012,Manhattan,Greenwich Village and Soho,40.7255,-73.9983
140,10021,Manhattan,Upper East Side,40.7685,-73.9588
141,10023,Manhattan,Upper West Side,40.7764,-73.9827
142,10026,Manhattan,Central Harlem,40.8019,-73.9531
143,10029,Manhattan,East Harlem,40.7918,-73.9447
144,10031,Manhattan,Inwood and Washington Heights,40.8246,-73.9507


In [21]:
df.shape

(43, 5)

In [22]:
import urllib

In [23]:
# The code was removed by Watson Studio for sharing.

In [24]:



def query_string_builder(groups=None,endpoints=None, args=None):
    query_string_list = []
     
    if args is not None and args.split('=')[1] == '': 
      s_parm = 'near=Toronto,ON' + '&' + 'query=vegetables' 
    else: 
      s_parm = args + '&' + 'query=vegetables'
                
                                
    allowed_groups = {'venues': lambda :str.format('venues') }
    regular_endpoints = {'search':{'head':'search',
                                   'tail':lambda x: str.format('search{}{}','&',x)}}
    if groups is not None and endpoints is not None:
        query_string_part1 = allowed_groups[groups]()+'/'+regular_endpoints[endpoints]['head']+ '?'
        query_string_list.append(query_string_part1)
        query_string_part2 = 'limit=1'+ '&'+ regular_endpoints[endpoints]['tail'](s_parm)
        query_string_list.append(query_string_part2)
                         
    return query_string_list



fs_query = query_string_builder(groups='venues',endpoints='search',args='l1=''')


fs_query



['venues/search?', 'limit=1&search&near=Toronto,ON&query=vegetables']

In [25]:
def credential_string_builder():
    return 'client_id=' + client_id + '&' + 'client_secret=' + client_secret + '&' + 'v=' + v + '&'

In [26]:
def four_square_uri_builder(credential_part = '',query_part1 = '',query_part2=''):
    base_uri_string = 'api.foursquare.com/v2/'
    return 'https://' + base_uri_string + query_part1 + credential_part + query_part2

In [28]:
import json
def query_four_square(f_uri = None):
    data = None
    try:
        json_data_file = urllib.request.urlopen(f_uri)
        json_data = json_data_file.read() 
        data = json.loads(json_data)
    except IOError as ioe:
        print(ioe)
    return data

In [29]:
query_p = query_string_builder(groups='venues',
                               endpoints='search',
                               args='l1=''')
fs_uri = four_square_uri_builder(credential_part = credential_string_builder(),
                                query_part1 = query_p[0],
                                query_part2 = query_p[1])

#fs_uri
data = query_four_square(f_uri = fs_uri)


data

{'meta': {'code': 200, 'requestId': '5dfafb47760a7f001b8bee40'},
 'response': {'venues': [{'id': '4d9219139d0f721e78632573',
    'name': "Phil's Place Fruit & Vegetables",
    'location': {'address': '93 Front St E',
     'crossStreet': 'Jarvis St',
     'lat': 43.64877916518546,
     'lng': -79.3715894297388,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64877916518546,
       'lng': -79.3715894297388}],
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['93 Front St E (Jarvis St)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1576729463',
    'hasPerk': False}],
  'geocode': {'what': '',
   'where': 't

In [30]:
js_data = data

In [31]:
from pandas.io.json import json_normalize
import re

df_fs = json_normalize(data = js_data)
#record_path = ['venues'],meta = ['meta','response'])
df_fs

    

,meta.code,meta.requestId,response.geocode.feature.cc,response.geocode.feature.displayName,response.geocode.feature.geometry.bounds.ne.lat,response.geocode.feature.geometry.bounds.ne.lng,response.geocode.feature.geometry.bounds.sw.lat,response.geocode.feature.geometry.bounds.sw.lng,response.geocode.feature.geometry.center.lat,response.geocode.feature.geometry.center.lng,...,response.geocode.feature.id,response.geocode.feature.longId,response.geocode.feature.matchedName,response.geocode.feature.name,response.geocode.feature.slug,response.geocode.feature.woeType,response.geocode.parents,response.geocode.what,response.geocode.where,response.venues
0,200,5dfafb47760a7f001b8bee40,CA,"Toronto, ON, Canada",43.855548,-79.104094,43.584643,-79.647455,43.70011,-79.4163,...,geonameid:6167865,72057594044095801,"Toronto, ON, Canada",Toronto,toronto,7,[],,toronto on,"[{'id': '4d9219139d0f721e78632573', 'name': 'P..."


In [32]:
columns = df_fs.columns
new_columns = []
for index in range(len(columns)):
    new_columns.append(re.search('[.]([a-zA-Z0-9]*)$',columns[index]).group(1))
df_fs.columns = new_columns

In [33]:
df_fs

,code,requestId,cc,displayName,lat,lng,lat,lng,lat,lng,...,id,longId,matchedName,name,slug,woeType,parents,what,where,venues
0,200,5dfafb47760a7f001b8bee40,CA,"Toronto, ON, Canada",43.855548,-79.104094,43.584643,-79.647455,43.70011,-79.4163,...,geonameid:6167865,72057594044095801,"Toronto, ON, Canada",Toronto,toronto,7,[],,toronto on,"[{'id': '4d9219139d0f721e78632573', 'name': 'P..."


In [34]:
df_fs1 = df_fs[['cc','name','lat','lng','venues']]

In [35]:
df_fs1

,cc,name,lat,lat,lat,lng,lng,lng,venues
0,CA,Toronto,43.855548,43.584643,43.70011,-79.104094,-79.647455,-79.4163,"[{'id': '4d9219139d0f721e78632573', 'name': 'P..."


In [38]:
for items in df_fs1['venues']:
    print(items)

[{'id': '4d9219139d0f721e78632573', 'name': "Phil's Place Fruit & Vegetables", 'location': {'address': '93 Front St E', 'crossStreet': 'Jarvis St', 'lat': 43.64877916518546, 'lng': -79.3715894297388, 'labeledLatLngs': [{'label': 'display', 'lat': 43.64877916518546, 'lng': -79.3715894297388}], 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['93 Front St E (Jarvis St)', 'Toronto ON', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d118951735', 'name': 'Grocery Store', 'pluralName': 'Grocery Stores', 'shortName': 'Grocery Store', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1576729463', 'hasPerk': False}]


In [39]:
# The code was removed by Watson Studio for sharing.

In [40]:
def route_query_string(from_ll:str,to_ll:str):
    base_string = "https://graphhopper.com/api/1/matrix?"
    
    cred_str = str.format('key={}',graphhopper_api_key)
    parm_str ='&type=json&vehicle=foot&debug=true&out_array=weights&out_array=times&out_array=distances&'
    query_string = base_string + 'point=' + from_ll + '&' + 'point=' + to_ll + parm_str + cred_str
    return query_string

In [41]:
q_s = route_query_string('43.855,-79.104','43.584,-79.647')
q_s

'https://graphhopper.com/api/1/matrix?point=43.855,-79.104&point=43.584,-79.647&type=json&vehicle=foot&debug=true&out_array=weights&out_array=times&out_array=distances&key=cd1e2904-8b4a-4b2f-88b4-5fb832e0ffce'

In [42]:
import urllib
import json

def query_graphhopper(query_uri):
    
    try:
        query_response_file = urllib.request.urlopen(query_uri)
        response_data = query_response_file.read() 
        jdata= json.loads(response_data)
        return jdata
    except IOError as ioe:
        print(ioe)
        return None

In [43]:
from pandas.io.json import json_normalize
js_data = query_graphhopper(q_s)
matrix_df = json_normalize(js_data)
js_data

{'distances': [[0, 62639], [62639, 0]],
 'times': [[0, 45099], [45099, 0]],
 'weights': [[0.0, 38610.656], [38610.656, 0.0]],
 'info': {'copyrights': ['GraphHopper', 'OpenStreetMap contributors']}}

In [44]:
matrix_df

,distances,info.copyrights,times,weights
0,"[[0, 62639], [62639, 0]]","[GraphHopper, OpenStreetMap contributors]","[[0, 45099], [45099, 0]]","[[0.0, 38610.656], [38610.656, 0.0]]"


In [50]:
matrix_df.drop(labels = ['info.copyrights'],inplace=True,axis =1)


In [55]:
matrix_df

,distances,times,weights
0,"[[0, 62639], [62639, 0]]","[[0, 45099], [45099, 0]]","[[0.0, 38610.656], [38610.656, 0.0]]"


In [51]:
# The code was removed by Watson Studio for sharing.

In [52]:
#q_s = route_query_string('43.855,-79.104','43.584,-79.647')
#q_s

In [53]:
# The code was removed by Watson Studio for sharing.

In [54]:
# The code was removed by Watson Studio for sharing.